# 形状変更

データの形状を変更する操作は、データ分析や加工において重要なステップです。Polarsでは効率的に形状を変更するための様々な方法を提供しています。この章では、その中でも特に重要な操作について説明します。

In [1]:
import polars as pl
from helper.jupyter import row

## ピボットテーブル

ピボットテーブルは、**データの集計や再構成**を行うための強力なツールです。特に、複数の次元（列）を指定してデータを再配置し、表形式で集計結果を分かりやすく整理するのに役立ちます。Polarsでは、`pivot`メソッドを使用してピボットテーブルを簡単に作成できます。一般的にピボットテーブルに変換すると、元々縦長いテーブルは横長いテーブルになります。また、`unpivot`メソッドは`pivot`の逆演算で、横長いテーブルを縦長いテーブルに変換します。

### pivot

`pivot`メソッドを使用すると、データフレームをピボットテーブルの形式に変換できます。この操作では、指定した列を基準にしてデータを集計し、行列形式で再構築します。

以下の例では、`name`列を新しい列ラベルとして使用し、`index`列を行ラベル（インデックス）として設定しています。また、`value`列の値を新しいテーブルのセルに挿入し、同じ行と列の組み合わせが複数存在する場合には`aggregate_function`で指定した方法（この場合は`first`）で値を集約します。

In [2]:
df = pl.DataFrame(
    dict(
        index=[0, 1, 2, 0, 1, 2, 0, 1, 2, 1],
        name=['A', 'A', 'A', 'B', 'B', 'B', 'C', 'C', 'C', 'A'],
        value=[10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
        score=[1, 3, 5, 4, 3, 2, 1, 2, 3, 4],
    )
)

df1 = df.pivot('name', index='index', values='value', aggregate_function='first')
row(df, df1)

index,name,value,score
i64,str,i64,i64
0,"""A""",10,1
1,"""A""",20,3
2,"""A""",30,5
0,"""B""",40,4
1,"""B""",50,3
2,"""B""",60,2
0,"""C""",70,1
1,"""C""",80,2
2,"""C""",90,3


`aggregate_function`引数を使用して、同じ行列の組み合わせに対応する複数の値をどのように集約するかを指定できます。Polarsはこの引数に以下のような標準的な関数を受け付けます：

1. **`sum`**: 値を合計します。
2. **`mean`**: 値の平均を計算します。
3. **カスタム演算式**: ユーザー定義の集約演算式を適用可能です。ここでは`pl.element()`ですべての値をリストに集約します。

以下の例を通して、異なる`aggregate_function`を使用したピボットの結果を確認します。

In [3]:
df2 = df.pivot('name', index='index', values='value', aggregate_function='sum')
df3 = df.pivot('name', index='index', values='value', aggregate_function='mean')
df4 = df.pivot('name', index='index', values='value', aggregate_function=pl.element())
row(df2, df3, df4)

index,A,B,C
i64,i64,i64,i64
0,10,40,70
1,120,50,80
2,30,60,90
index,A,B,C
i64,f64,f64,f64
0,10.0,40.0,70.0
1,60.0,50.0,80.0
2,30.0,60.0,90.0
index,A,B,C


`values`引数に複数の列を渡すことで、指定した複数の列がピボットテーブルの中で展開されます。この場合、結果のテーブルでは、列ラベルが`name`列の値に基づき、各ラベルに対応するサブ列として指定された値の列が表示されます。又、`separator`引数を指定することで、列名と値の結合文字を自由に変更できます。

In [4]:
df5 = df.pivot('name', index='index', values=['value', 'score'], aggregate_function=pl.element())
df6 = df.pivot('name', index='index', values=['value', 'score'], aggregate_function="max", separator=":")
row(df5, df6)

index,value_A,value_B,value_C,score_A,score_B,score_C
i64,list[i64],list[i64],list[i64],list[i64],list[i64],list[i64]
0,[10],[40],[70],[1],[4],[1]
1,"[20, 100]",[50],[80],"[3, 4]",[3],[2]
2,[30],[60],[90],[5],[2],[3]
index,value:A,value:B,value:C,score:A,score:B,score:C
i64,i64,i64,i64,i64,i64,i64
0,10,40,70,1,4,1
1,100,50,80,4,3,2
2,30,60,90,5,2,3


### unpivot

`DataFrame.unpivot()`は`pivot()`の逆操作で、横長のデータフレームを縦長の形式に変換します。このメソッドを使うことで、複数の列を1つの列に変換し、元の形式に戻すことができます。`unpivot`では、指定した列を一つの列にまとめ、その他の列をインデックスとして保持します。これにより、カテゴリごとにデータを整形することができます。`unpivot()`メソッドには以下の引数があります：

- **`on`**: unpivotする対象となる列を指定します。列名や列名のリスト、あるいは列を選択するセレクターを使用できます。通常、`cs.exclude()`を使ってインデックス列や不要な列を除外することが一般的です。列を選択する演算式（`pl.col()`や`pl.exclude()`など）は使用できません。
  
- **`index`**: 新しい縦長データフレームで、インデックスとして使用する列を指定します。`index`列は、元のデータフレームでの位置を保持するための役割を果たします。

- **`variable_name`**: `on`引数で指定した複数の列名から一つの列を作成します。この引数で、この新しい列の名前を指定します。例えば、元の列名をまとめた新しい列を作成できます。

- **`value_name`**: `on`で指定した列の値から作成した列の名前を指定します。これにより、`on`で指定された各列の値が新しい列に変換され、結果として縦長のデータフレームに格納されます。

次の例では、`index`列を除くすべての列を`unpivot`し、`name`列に元の列名を、`value`列に対応する値を格納します。`index`列はそのまま保持され、結果的に縦長のデータフレームが作成されます。このように、`unpivot`を使うと、ピボットで横長に変換されたデータを再び縦長に戻すことができます。

In [5]:
from polars import selectors as cs
df_u1 = df1.unpivot(on=cs.exclude('index'), index='index', variable_name='name', value_name='value')
row(df1, df_u1)

shape: (3, 4) index A B C i64 i64 i64 i64 0 10 40 70 1 20 50 80 2 30 60 90 
 shape: (9, 3) index name value i64 str i64 0 "A" 10 1 "A" 20 2 "A" 30 0 "B" 40 1 "B" 50 2 "B" 60 0 "C" 70 1 "C" 80 2 "C" 90

`df6`のような複数の列を`pivot()`の`values`引数に渡して作成したデータフレームを元の形に戻すには少し複雑な処理が必要です。以下のコードでは、`unpivot()`を使って縦長のデータフレームに戻し、その後、列名と元の名前を分割して`pivot()`を使って再構成しています。

1. **`unpivot(on=cs.exclude('index'), index='index')`**:
   - `df6`のピボットされたデータフレームを縦長のデータフレームに変換します。
   - `index='index'`で、`index`列はそのまま保持され、`on=cs.exclude('index')`により、`index`列以外の全ての列が縦長になります。`variable`列には元々の列名（例えば、`value:A`, `score:B`など）が格納されます。

2. **`pl.col('variable').str.split(":")`**:
   - `variable`列（元々の列名）を`:`で分割し、リスト（2つの要素）にします。例えば、`value:A`は`['value', 'A']`に分割されます。

3. **`.list.to_struct(fields=['colname', 'name'])`**:
   - 分割されたリストを構造体に変換します。この構造体は2つのフィールド（`colname`と`name`）を持ちます。

4. **`.struct.unnest()`**:
   - 構造体を展開し、2つの新しい列（`colname`と`name`）を作成します。これにより、`colname`列には元の列名（`value`、`score`）が、`name`列には元々のデータフレームのこの二つ列のデータが格納されます。

5. **`pivot('colname', index=['index', 'name'], values='value')`**:
   - 最後に、`colname`列を使って元のデータをピボットします。`index`には元々の`index`列と`name`列を使い、`values`には元々の値が入っていた`value`列を指定します。

In [6]:
(
df6.unpivot(on=cs.exclude('index'), index='index')
.with_columns(
    pl.col('variable')
      .str.split(":")
      .list.to_struct(fields=['colname', 'name'])
      .struct.unnest()
)
.pivot('colname', index=['index', 'name'], values='value')
)

index,name,value,score
i64,str,i64,i64
0,"""A""",10,1
1,"""A""",100,4
2,"""A""",30,5
0,"""B""",40,4
1,"""B""",50,3
2,"""B""",60,2
0,"""C""",70,1
1,"""C""",80,2
2,"""C""",90,3


## unstack

`unstack()`はNumPyの`reshape`に似ており、一つの列の要素を複数の列に展開するために使用します。`how`引数で並び順を指定でき、`'vertical'`では縦方向（行優先）、`'horizontal'`では横方向（列優先）に展開されます。また、`fill_values`引数を使うことで、欠損部分に埋める値を指定できます。

以下の例では、`index`、`name`、`value`、`score`の4つの列を持つデータフレームを作成し、`unstack()`を使って縦方向および横方向に展開します。

In [7]:
df_vertical = df.unstack(4, how='vertical', columns=['index', 'name'], fill_values=[-1, '?'])
df_horizontal = df.unstack(3, how='horizontal', columns='index')
row(df, df_vertical, df_horizontal)

index,name,value,score
i64,str,i64,i64
0,"""A""",10,1
1,"""A""",20,3
2,"""A""",30,5
0,"""B""",40,4
1,"""B""",50,3
2,"""B""",60,2
0,"""C""",70,1
1,"""C""",80,2
2,"""C""",90,3


## to_dummies

`to_dummies()` は、カテゴリカルデータを**ダミー変数（one-hotエンコーディング）** に変換するためのメソッドです。指定した列に含まれる各カテゴリが新しい列として展開され、その列には **0** または **1** の値が割り当てられます。  

- **0**: そのカテゴリに該当しない場合  
- **1**: そのカテゴリに該当する場合  

複数の列を指定した場合、それぞれの列に対してダミー変数を生成します。

- `drop_first=False`（デフォルト）: すべてのカテゴリをダミー変数として作成します。
- `drop_first=True`: **基準となるカテゴリ**（最初のカテゴリ）を除外し、残りのカテゴリのみをダミー変数として作成します。これは**多重共線性**を防ぐために使われます（回帰分析などの統計モデルで役立ちます）。

In [8]:
df_original = df.filter(pl.col('index') < 2).select('index', 'name', 'value')
df_dummies1 = df_original.to_dummies(['index', 'name'], drop_first=False)
df_dummies2 = df_original.to_dummies(['index', 'name'], drop_first=True)
row(df_original, df_dummies1, df_dummies2)

shape: (7, 3) index name value i64 str i64 0 "A" 10 1 "A" 20 0 "B" 40 1 "B" 50 0 "C" 70 1 "C" 80 1 "A" 100 
 shape: (7, 6) index_0 index_1 name_A name_B name_C value u8 u8 u8 u8 u8 i64 1 0 1 0 0 10 0 1 1 0 0 20 1 0 0 1 0 40 0 1 0 1 0 50 1 0 0 0 1 70 0 1 0 0 1 80 0 1 1 0 0 100 
 shape: (7, 4) index_1 name_B name_C value u8 u8 u8 i64 0 0 0 10 1 0 0 20 0 1 0 40 1 1 0 50 0 0 1 70 1 0 1 80 1 0 0 100